<a href="https://colab.research.google.com/github/greyhound101/Multihead_attention/blob/master/sesqueezenet_1_2413.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [2]:
pip install git+https://github.com/rcmalli/keras-squeezenet.git

  Cloning https://github.com/rcmalli/keras-squeezenet.git to /tmp/pip-req-build-uw_vr2gt
  Running command git clone -q https://github.com/rcmalli/keras-squeezenet.git /tmp/pip-req-build-uw_vr2gt


In [3]:
!nvidia-smi

Thu Dec 16 16:59:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
import zipfile
import h5py
import cv2
from matplotlib import pyplot as plt
import glob, os
from matplotlib import pyplot as plt
import h5py
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm
import time
import gc
import pandas as pd
import imgaug as ia
from imgaug import augmenters as iaa

import zipfile
import h5py
import cv2
from matplotlib import pyplot as plt
import glob, os
from matplotlib import pyplot as plt
import h5py
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm
import time
import gc


In [5]:
path = F"/content/gdrive/My Drive/check.npy" 
df=np.load(path,allow_pickle=True)
df=df.item()

In [6]:

#shuffle samples
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]



#change targets
def change(img):
    resized = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA )
    return resized




#get train and test splits
def get_trn_tst(df,tst_fold):
  idx=np.asarray(df['fold'])
  y=np.asarray(df['label'])
  y-=1
  img=np.asarray(df['image'])
  img1=[]
  for i in range(len(img)):
        img1.append(change(img[i]))
  img1=np.asarray(img1)
  del([img])
  gc.collect()
  trn_y=np.asarray(y[(idx!=tst_fold)])
  trn_img=np.asarray(img1[(idx!=tst_fold)])
  tst_y=np.asarray(y[(idx==tst_fold)])
  tst_img=img1[idx==tst_fold]
  trn_img=np.repeat(trn_img.reshape((trn_img.shape[0],224,224,1)),3,axis=3)
  tst_img=np.repeat(tst_img.reshape((tst_img.shape[0],224,224,1)),3,axis=3)
  return (trn_img.copy(),trn_y.copy()),(tst_img.copy(),tst_y.copy())

In [7]:
import zipfile
import h5py
import cv2
import glob, os
from matplotlib import pyplot as plt
import h5py
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm
import time
import gc
import keras
from keras.activations import softmax
from keras import backend as K
from keras.layers import *

In [8]:
# from keras_applications.imagenet_utils import _obtain_input_shape
from keras import backend as K
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.models import Model
# from keras.engine.topology import get_source_inputs
from tensorflow.keras.utils import get_file,get_source_inputs
# from keras.utils import layer_utils


sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu1 = "relu_"

WEIGHTS_PATH = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_NO_TOP = "https://github.com/rcmalli/keras-squeezenet/releases/download/v1.0/squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5"
def squeeze_excite_block(input_tensor, ratio=16):
    """ Create a channel-wise squeeze-excite block
    Args:
        input_tensor: input Keras tensor
        ratio: number of output filters
    Returns: a Keras tensor
    References
    -   [Squeeze and Excitation Networks](https://arxiv.org/abs/1709.01507)
    """
    init = input_tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = K.int_shape(init)[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu1 + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu1 + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu1 + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x


# Original SqueezeNet from paper.

def SqueezeNet(include_top=True, weights='imagenet',
               input_tensor=None, input_shape=(224,224,3),
               pooling=None,
               classes=1000):
    """Instantiates the SqueezeNet architecture.
    """
        
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `include_top`'
                         ' as true, `classes` should be 1000')


    input_shape = input_shape

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor


    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    if include_top:
        # It's not obvious where to cut the network... 
        # Could do the 8th or 9th layer... some work recommends cutting earlier layers.
    
        x = Dropout(0.5, name='drop9')(x)

        x = Convolution2D(classes, (1, 1), padding='valid', name='conv10')(x)
        x = Activation('relu', name='relu_conv10')(x)
        x = GlobalAveragePooling2D()(x)
        x = Activation('softmax', name='loss')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling=='max':
            x = GlobalMaxPooling2D()(x)
        elif pooling==None:
            pass
        else:
            raise ValueError("Unknown argument for 'pooling'=" + pooling)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    model = Model(inputs, x, name='squeezenet')

    # load weights
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models')
        else:
            weights_path = get_file('squeezenet_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models')
            
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model


In [9]:
mod=SqueezeNet()

In [10]:
# from tensorflow import keras
# from keras.activations import softmax
# from tensorflow.keras import backend as K
# import tensorflow as tf
# class LayerNormalization(Layer):
#     def __init__(self, eps=1e-6, **kwargs):
#         self.eps = eps
#         super(LayerNormalization, self).__init__(**kwargs)
#     def build(self, input_shape):
#         self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
#                                      initializer=Ones(), trainable=True)
#         self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
#                                     initializer=Zeros(), trainable=True)
#         super(LayerNormalization, self).build(input_shape)
#     def call(self, x):
#         mean = K.mean(x, axis=-1, keepdims=True)
#         std = K.std(x, axis=-1, keepdims=True)
#         return self.gamma * (x - mean) / (std + self.eps) + self.beta
#     def compute_output_shape(self, input_shape):
#         return input_shape
# class abc(Layer):
#     def __init__(self,inr,size,mo,up,**kwargs):
#         super(abc, self).__init__(**kwargs)
#         self.inr=inr
#         self.mo=mo
#         self.up=up
#         self.size=size
#     def get_config(self):
#         base_config = super(abc, self).get_config()

#     def build(self, input_shape):
#         super(abc, self).build(input_shape)
#         self.cv1 = Conv2D(self.inr,1)
#         self.cv2 = Conv2D(self.inr,1)
#         self.up = UpSampling2D(interpolation='bilinear',size=(self.up,self.up))
#         self.dns1=Dense(1)
#     def call(self, img,y):
#         y = self.cv1(y)
#         x = self.cv2(img)
#         y = self.up(y)
        
#         x = Add()([y,x])
#         x = ReLU()(x)
#         x = K.mean(x,axis=-1)
#         x = Reshape((self.size,self.size,1))(x)
        
#         map = softmax(x,axis=[2,3])


#         return tf.math.multiply(img,map)

# class SpatialGate(keras.layers.Layer):
#     def __init__(self,**kwargs):
#         super(SpatialGate, self).__init__(**kwargs)

#     def get_config(self):
#         base_config = super(SpatialGate, self).get_config()

#     def build(self, input_shape):
#         super(SpatialGate, self).build(input_shape)
#         self.cv2 = Conv2D(1,1)
#     def call(self, img):
        
#         img_avg = K.expand_dims(K.mean(img,-1),-1)
#         img_max = K.expand_dims(K.max(img,-1),-1)
#         total = Concatenate(-1)([img_avg,img_max])
#         x = self.cv2(total)
#         x = keras.activations.sigmoid(x)

#         return tf.math.multiply(img,x)
# class ChannelGate(keras.layers.Layer):
#     def __init__(self,inr,ratio,**kwargs):
#         super(ChannelGate, self).__init__(**kwargs)
#         self.inr=inr
#         self.ratio=ratio

#     def get_config(self):
#         base_config = super(abc, self).get_config()

#     def build(self, input_shape):
#         super(ChannelGate, self).build(input_shape)
#         self.dns1 = Dense(self.inr/self.ratio,activation='relu')
#         self.dns2 = Dense(self.inr)
#         self.spt = SpatialGate()
#     def call(self, img):
        
#         img_avg = self.dns2(self.dns1(GlobalAveragePooling2D()(img)))
#         img_max = self.dns2(self.dns1(GlobalMaxPooling2D()(img)))
#         x = keras.activations.sigmoid(img_max+img_avg)
#         x = Reshape((1,1,self.inr))(x)

#         return self.spt(tf.math.multiply(img,x))

# from tensorflow import keras 
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.layers import *
# from tensorflow.keras.applications import *
# def load_model():   
  
#   K.clear_session() 
#   mod=densenet.DenseNet121(include_top=True, weights='imagenet')
#   out_1=mod.layers[-2].output
#   out=Dense(3,activation='softmax')(out_1)
#   model=Model(inputs=mod.input,outputs=out)
#   return model


# from tensorflow import keras 
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.layers import *
# from tensorflow.keras.applications import *
# from tensorflow import keras 
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.layers import *
# from tensorflow.keras.applications import *
# def load_model():   
  
#   K.clear_session() 
#   mod=SqueezeNet(include_top=True, weights='imagenet')
#   d = mod.get_layer('relu_conv10').output
#   d = Conv2D(512,1)(d)
#   a = mod.get_layer('fire5/concat').output
#   a = Conv2D(256,2)(a)
  
#   y = Conv2D(16,1)(a)
#   x = Conv2D(16,1)(d)
#   x = UpSampling2D(size=(2,2))(x)
#   x = Add()([y,x])
#   x = Lambda(lambda x: relu(x))(x)
#   x = Lambda(lambda x: K.mean(x,axis=-1))(x)
#   first = Reshape((26,26,-1))(x)

#   a = mod.get_layer('fire3/concat').output
#   a = Conv2D(256,5)(a)
#   y = Conv2D(16,1)(a)
#   x = Conv2D(16,1)(d)
#   x = UpSampling2D(size=(4,4))(x)
#   x = Add()([y,x])
#   x = Lambda(lambda x: relu(x))(x)
#   x = Lambda(lambda x: K.mean(x,axis=-1))(x)
#   second = Reshape((52,52,-1))(x)
#   conc=Concatenate(axis=-1)([a,first,second])
#   conc = GlobalMaxPooling2D()(conc)

#   conc = Dense(3, activation="softmax")(conc) 
  

#   mod=Model(mod.inputs,conc)
#   return mod



# import keras
# import pandas as pd
# from keras.applications.nasnet import NASNetMobile, preprocess_input
import imgaug as ia
from imgaug import augmenters as iaa

def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result
def Hflip( images):
		seq = iaa.Sequential([iaa.Fliplr(1.0)])
		return seq.augment_images(images)
def Vflip( images):
		seq = iaa.Sequential([iaa.Flipud(1.0)])
		return seq.augment_images(images)
def noise(images):
    ls=[]
    for i in images:
        x = np.random.normal(loc=0, scale=0.05, size=(299,299,3))
        ls.append(i+x)
    return ls
def rotate(images):
    ls=[]
    for angle in range(-15,20,5):
        for image in images:
            ls.append(rotate_image(image,angle))
    return ls
import tensorflow as tf
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, images, labels, batch_size=64, image_dimensions = (96 ,96 ,3), shuffle=False, augment=False):
    self.labels       = labels              # array of labels
    self.images = images        # array of image paths
    self.batch_size   = batch_size          # batch size
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(self.labels.shape[0] / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(self.labels.shape[0])

  def __getitem__(self, index):
		# selects indices of data for next batch
    indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    # select data and load images
    labels = self.labels.loc[indexes]
    img = [self.images[k].astype(np.float32) for k in indexes]
    imgH=Hflip(img)
    imgV=Vflip(img)
    imgR=rotate(img)
    images=[]
    images.extend(imgH)
    images.extend(imgV)
    images.extend(imgR)
    lbl=labels.copy()
    labels=pd.DataFrame()
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    #images = np.array([preprocess_input(img) for img in images])
    return np.asarray(images), labels.values


best_accuracy_last={}
final_accuracy_last={}
history_last={}
answers_last={}
predictions_last={}
predictions_last_best={}
times_last={}


#shuffle samples
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]



#change targets
def change(img):
    resized = cv2.resize(img, (227,227), interpolation = cv2.INTER_AREA )
    return resized




#get train and test splits
def get_trn_tst(df,tst_fold):
  idx=np.asarray(df['fold'])
  y=np.asarray(df['label'])
  y-=1
  img=np.asarray(df['image'])
  img1=[]
  for i in range(len(img)):
        img1.append(change(img[i]))
  img1=np.asarray(img1)
  del([img])
  gc.collect()
  trn_y=np.asarray(y[(idx!=tst_fold)])
  trn_img=np.asarray(img1[(idx!=tst_fold)])
  tst_y=np.asarray(y[(idx==tst_fold)])
  tst_img=img1[idx==tst_fold]
  trn_img=np.repeat(trn_img.reshape((trn_img.shape[0],224,224,1)),3,axis=3)
  tst_img=np.repeat(tst_img.reshape((tst_img.shape[0],224,224,1)),3,axis=3)
  return (trn_img.copy(),trn_y.copy()),(tst_img.copy(),tst_y.copy())

In [11]:

def upd(dk,data):
    if dk==0:
        dk=data
    else:
        for ky in data.keys():
            dk[ky].extend(data[ky])
    return dk

In [12]:
from tensorflow.keras.applications.densenet import *
def load_model():   
  K.clear_session() 

  mod1=SqueezeNet(input_shape=(224,224,3))
  out_1=mod1.layers[-3].output
  out_1 = squeeze_excite_block(out_1)
  out_1 = GlobalAveragePooling2D()(out_1)
  out=Dense(3,activation='softmax')(out_1)
  model=Model(inputs=mod1.input,outputs=out)

  return model

In [13]:
  from tensorflow.keras.optimizers import Adam
  index=1
  best_accuracy_last={}
  final_accuracy_last={}
  history_last={}
  answers_last={}
  predictions_last={}
  predictions_last_best={}
  times_last={}
  epoch=50
  pre_acc=0
  best=0
  fold='fold_'+str(index)
  trn,tst=get_trn_tst(df,index)
  history_last[fold]=0



  plt.imshow(trn[0][0])
  plt.show()
  plt.imshow(tst[0][0])
  plt.show()



  trn_x,trn_y=unison_shuffled_copies(trn[0],trn[1])
  tst_x,tst_y=unison_shuffled_copies(tst[0],tst[1])



  model=load_model()


  
  #compiling the model
  model.compile(optimizer=Adam(2e-4,decay=1e-3), 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
  train_data = DataGenerator(trn_x,pd.get_dummies(trn_y), batch_size=4, augment=True)
  ln=len(trn_y)
  # del([trn_x,trn_y,trn,tst])
  # gc.collect()
  #fitting the model
  #timing
  start=time.time()
  print('training')
  # hist=model.fit_generator(train_data,epochs=50,steps_per_epoch=ln//4,verbose=1)
  model.load_weights('/content/gdrive/My')
  history_last[fold]=upd(history_last[fold],hist.history)

  end=time.time()
  times_last[fold]=end-start

  #getting the prediction 
  pre=model.predict(tst_x)
  



  #select the maximum position
  pre=np.argmax(pre,1)
  predictions_last[fold]=pre

  
  
  
  #getting the accuracy
  new_acc=accuracy_score(pre,tst_y)

  


  #storing the predictions
  final_accuracy_last[fold]=new_acc








  #storing the answers
  answers_last[fold]=tst_y
    
    
    
    
  #freeing memory
  del([tst_x,tst_y])
  gc.collect()

  plt.plot(history_last[fold]['loss'])
  plt.show()
  print(new_acc)
  index=str(index)
  type='squeezenet_'
  model1='SE'
  path='/content/gdrive/My Drive/'
  np.save(path+"/best_accuracy_all_fold_"+index+"_"+model1+"_"+type+".npy",best_accuracy_last)
  np.save(path+'/final_accuracy_all_fold'+index+"_"+model1+"_"+type+".npy",final_accuracy_last)
  np.save(path+'/history_all_fold_'+index+"_"+model1+"_"+type+".npy",history_last)
  np.save(path+'/answers_all_fold_'+index+"_"+model1+"_"+type+".npy",answers_last)
  np.save(path+'/predictions_all_fold_'+index+"_"+model1+"_"+type+".npy",predictions_last)
  np.save(path+'/predictions_all_best_fold_'+index+"_"+model1+"_"+type+".npy",predictions_last_best)
  np.save(path+'/times_all_fold_'+index+"_"+model1+"_"+type+".npy",times_last)
  model.save_weights(path+type+index+'.hdf5')

ValueError: ignored